In [1]:
from TrainDataPreprocess import ProcessData
from ModelSet import NERModel, CategoriesClassifier, PolaritiesClassifier
from GetData import GetData

train_reviews_path = '../data/Train_reviews.csv'
train_labels_path = '../data/Train_labels.csv'
test_reviews_path = '../data/Test_reviews.csv'

/Users/zhaoliang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#预处理数据
process_data = ProcessData(train_reviews_path, train_labels_path)
process_data.sava_data()

#获取训练数据和测试数据
get_data_obj = GetData('word')
chunk2id, id2chunk, vocab2id, id2vocab, ner_data, ner_label = get_data_obj.get_ner_train_data()
content_data, aspect_data, opinion_data, category_label, id2categories = get_data_obj.get_categories_train_data()
content_data, aspect_data, opinion_data, polarity_label, id2polarities = get_data_obj.get_polarities_train_data()
ids, data, sents = get_data_obj.get_test_ner_data(test_reviews_path, vocab2id)

Building prefix dict from the default dictionary ...
I0826 10:25:45.078475 4458923456 __init__.py:111] Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/x8/4yf3l2h548b6w9d_ym2c1lvr0000gp/T/jieba.cache
I0826 10:25:45.079925 4458923456 __init__.py:131] Loading model from cache /var/folders/x8/4yf3l2h548b6w9d_ym2c1lvr0000gp/T/jieba.cache


数据预处理...
数据表整合完成...


Loading model cost 0.650 seconds.
I0826 10:25:45.729952 4458923456 __init__.py:163] Loading model cost 0.650 seconds.
Prefix dict has been built succesfully.
I0826 10:25:45.731043 4458923456 __init__.py:164] Prefix dict has been built succesfully.


字典生成与存储...
字典生成与存储完毕
数据生成与存储...
数据生成与存储完毕！


In [3]:
max_sent_len = 0
for item in content_data:
    if len(item) > max_sent_len:
        max_sent_len = len(item)
for item in data:
    if len(item) > max_sent_len:
        max_sent_len = len(item)

In [4]:
ner = NERModel(maxlen = max_sent_len, word_dict_size = len(vocab2id), word_vec_size = 128, class_label_count = len(chunk2id))
ner.train(ner_data, ner_label )
ner_result = ner.predict(data, id2chunk)

W0826 10:25:54.582780 4458923456 deprecation_wrapper.py:119] From /Users/zhaoliang/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0826 10:25:54.596755 4458923456 deprecation_wrapper.py:119] From /Users/zhaoliang/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0826 10:25:54.607737 4458923456 deprecation_wrapper.py:119] From /Users/zhaoliang/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0826 10:25:55.185096 4458923456 deprecation_wrapper.py:119] From /Users/zhaoliang/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_w

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     (None, 47)                0         
_________________________________________________________________
embedding_layer (Embedding)  (None, 47, 128)           387968    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 47, 64)            41216     
_________________________________________________________________
dropout_1 (Dropout)          (None, 47, 64)            0         
_________________________________________________________________
crf_1 (CRF)                  (None, 47, 5)             360       
Total params: 429,544
Trainable params: 429,544
Non-trainable params: 0
_________________________________________________________________


W0826 10:25:57.424808 4458923456 deprecation_wrapper.py:119] From /Users/zhaoliang/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 2906 samples, validate on 323 samples
Epoch 1/500
2906/2906 [==============================] - 8s 3ms/step - loss: 0.4539 - crf_viterbi_accuracy: 0.8668 - val_loss: 0.2349 - val_crf_viterbi_accuracy: 0.9177
Epoch 2/500
2906/2906 [==============================] - 6s 2ms/step - loss: 0.2084 - crf_viterbi_accuracy: 0.9208 - val_loss: 0.1604 - val_crf_viterbi_accuracy: 0.9424
Epoch 3/500
2906/2906 [==============================] - 6s 2ms/step - loss: 0.1436 - crf_viterbi_accuracy: 0.9478 - val_loss: 0.1222 - val_crf_viterbi_accuracy: 0.9592
Epoch 4/500
2906/2906 [==============================] - 6s 2ms/step - loss: 0.1091 - crf_viterbi_accuracy: 0.9625 - val_loss: 0.1027 - val_crf_viterbi_accuracy: 0.9675
Epoch 5/500
2906/2906 [==============================] - 6s 2ms/step - loss: 0.0896 - crf_viterbi_accuracy: 0.9700 - val_loss: 0.0927 - val_crf_viterbi_accuracy: 0.9708
Epoch 6/500
2906/2906 [==============================] - 6s 2ms/step - loss: 0.0777 - crf_viterbi_accuracy: 

In [5]:
from GenSubmit import GenSubmit
gs = GenSubmit(list(process_data.train_reviews['Reviews']) + sents, id2vocab)

In [6]:
ner_table = gs.get_ner_result(ids, sents, data, ner_result)
ret_content_arr, ret_aspect_arr, ret_opinion_arr = get_data_obj.get_test_category_polarity_data(ner_table, vocab2id)

40
双十一买的73 120ml特别划得来，希望以后能多多做活动。
[]
['特别UNK']
********************
225
用了感觉脸上闷闷的
[]
['UNK的']
********************
242
第一次买也是最后一次买，除了便宜...
[]
['便宜UNK']
********************
397
补水功能强大，跟绵
[]
['补水UNK']
********************
464
用几天了，感觉不像正品。液体太稀
[]
['UNK正品']
********************
581
收到宝贝打开一看，挺不错身，不过过没用，等用过再来追评。
[]
['挺不错UNK']
********************
1233
外面有点脏，打开盖，霜都泄露了(눈_눈)
[]
['有点UNK']
********************
1762
这个对于我这种干皮来书就是大救星，很细润，如果你也和我一样是很干的皮肤的话就可以用来试试
[]
['很UNK']
********************


In [7]:
len(ner_result)

2237

In [8]:
ner_table.head()

,id,Reviews,AspectTerms,A_start,A_end,OpinionTerms,O_start,O_end
0,1,最近太忙一直没有空来评价，东西已试过是正品，擦在脸上勾称白嫩，是个不错的商品,_,,,是正品,18,21
1,1,最近太忙一直没有空来评价，东西已试过是正品，擦在脸上勾称白嫩，是个不错的商品,_,,,白嫩,28,30
2,1,最近太忙一直没有空来评价，东西已试过是正品，擦在脸上勾称白嫩，是个不错的商品,_,,,不错,33,35
3,2,这款气垫，不是我喜欢的，因为颜色明显，还有让人家一眼就看出,颜色,14,16,_,,
4,2,这款气垫，不是我喜欢的，因为颜色明显，还有让人家一眼就看出,_,,,不是,5,7


In [9]:
aspect_max_len = 0
for item in aspect_data:
    if len(item) > aspect_max_len:
        aspect_max_len = len(item)
for item in ret_aspect_arr:
    if len(item) > aspect_max_len:
        aspect_max_len = len(item)
        
option_max_len = 0
for item in opinion_data:
    if len(item) > option_max_len:
        option_max_len = len(item)
for item in ret_opinion_arr:
    if len(item) > option_max_len:
        option_max_len = len(item)

In [10]:
for i in range(len(aspect_data)):
    for k in range(len(aspect_data[i])):
        if aspect_data[i][k] is None:
            print(aspect_data[i])
            print(1)

In [11]:
cc = CategoriesClassifier(content_max_len = max_sent_len, aspect_max_len=aspect_max_len, option_max_len=option_max_len, categories_num=len(id2categories), word_dict_size=len(vocab2id), word_vec_size=128)
cc.train(content_data, aspect_data, opinion_data, category_label)


/Users/zhaoliang/gitProject/zhijiang/code/ModelSet.py:175: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(filters=50, kernel_size=5, padding="valid")`
  conv_layer = Conv1D(nb_filter=50, filter_length=(2 * half_window_size + 1), border_mode='valid')
W0826 10:28:07.493206 4458923456 deprecation_wrapper.py:119] From /Users/zhaoliang/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3980: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.

/Users/zhaoliang/gitProject/zhijiang/code/ModelSet.py:188: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  model = Model(input=[content_input_layer, aspect_input_layer, option_input_layer], output=output)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
content_input_layer (InputLayer (None, 47)           0                                            
__________________________________________________________________________________________________
aspect_input_layer (InputLayer) (None, 4)            0                                            
__________________________________________________________________________________________________
option_input_layer (InputLayer) (None, 6)            0                                            
__________________________________________________________________________________________________
embedding_layer (Embedding)     multiple             387968      content_input_layer[0][0]        
                                                                 aspect_input_layer[0][0]         
          

In [12]:
pc = PolaritiesClassifier(content_max_len = max_sent_len, aspect_max_len=aspect_max_len, option_max_len=option_max_len, word_dict_size=len(vocab2id), word_vec_size=128)
pc.train(content_data, aspect_data, opinion_data, polarity_label)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
content_input_layer (InputLayer (None, 47)           0                                            
__________________________________________________________________________________________________
aspect_input_layer (InputLayer) (None, 4)            0                                            
__________________________________________________________________________________________________
option_input_layer (InputLayer) (None, 6)            0                                            
__________________________________________________________________________________________________
embedding_layer (Embedding)     multiple             387968      content_input_layer[0][0]        
                                                                 aspect_input_layer[0][0]         
          

/Users/zhaoliang/gitProject/zhijiang/code/ModelSet.py:99: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(filters=50, kernel_size=5, padding="valid")`
  conv_layer = Conv1D(nb_filter=50, filter_length=(2 * half_window_size + 1), border_mode='valid')
/Users/zhaoliang/gitProject/zhijiang/code/ModelSet.py:112: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  model = Model(input=[content_input_layer, aspect_input_layer, option_input_layer], output=output)


Train on 5969 samples, validate on 664 samples
Epoch 1/500
5969/5969 [==============================] - 3s 459us/step - loss: 0.2360 - acc: 0.9186 - val_loss: 0.0953 - val_acc: 0.9654
Epoch 2/500
5969/5969 [==============================] - 2s 342us/step - loss: 0.0850 - acc: 0.9702 - val_loss: 0.0791 - val_acc: 0.9729
Epoch 3/500
5969/5969 [==============================] - 2s 348us/step - loss: 0.0550 - acc: 0.9826 - val_loss: 0.0749 - val_acc: 0.9759
Epoch 4/500
5969/5969 [==============================] - 2s 353us/step - loss: 0.0403 - acc: 0.9879 - val_loss: 0.0765 - val_acc: 0.9744
Epoch 5/500
5969/5969 [==============================] - 2s 343us/step - loss: 0.0313 - acc: 0.9896 - val_loss: 0.1035 - val_acc: 0.9639
Epoch 6/500
5969/5969 [==============================] - 2s 342us/step - loss: 0.0255 - acc: 0.9915 - val_loss: 0.0865 - val_acc: 0.9684
Epoch 7/500
5969/5969 [==============================] - 2s 340us/step - loss: 0.0189 - acc: 0.9945 - val_loss: 0.0899 - val_acc: 0

In [13]:
ret_aspect_arr[0]

[0]

In [14]:
cc_result = cc.predict(ret_content_arr, ret_aspect_arr, ret_aspect_arr, id2categories)
pc_result = pc.predict(ret_content_arr, ret_aspect_arr, ret_opinion_arr)
ner_table['Categories'] = cc_result
ner_table['Polarities'] = pc_result
final_table = ner_table[['id', 'AspectTerms', 'OpinionTerms', 'Categories', 'Polarities']]
final_table.to_csv('../result/2019-08-26.csv', header=None, index=False)

In [15]:
final_table.head()

,id,AspectTerms,OpinionTerms,Categories,Polarities
0,1,_,是正品,使用体验,正面
1,1,_,白嫩,使用体验,正面
2,1,_,不错,使用体验,正面
3,2,颜色,_,功效,负面
4,2,_,不是,使用体验,负面


In [16]:
len(list(set(final_table['id'])))

2237

In [17]:
len(list(set(ner_table['id'])))

2237

In [18]:
import pandas as pd
table = pd.read_csv(test_reviews_path)

In [19]:
len(list(set(table['id'])))

2237

In [20]:
for item in list(set(table['id'])):
    if item not in list(set(final_table['id'])):
        print(item)